In [1]:
import comet_ml
import numpy as np
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
import torch
import torch.nn as nn
import torch.nn.functional as F

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-12 17:16:58.245064: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-12 17:16:58.257043: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-12 17:16:58.269997: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-12 17:16:58.273877: E external/l

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the STS-B dataset
dataset = load_dataset("glue", "stsb")

# Load the DistilBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Preprocess the dataset: tokenize the sentence pairs
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['sentence1', 'sentence2', 'idx'])
tokenized_datasets = tokenized_datasets.map(lambda examples: {'label': float(examples['label'])})


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
from transformers import DistilBertForSequenceClassification, AutoModelForSequenceClassification, DistilBertConfig, DataCollatorWithPadding
from iDistilbert import iDistilBertForSequenceClassification
student_config = DistilBertConfig(
    output_hidden_states = False,
    distance_metric = "manhattan_distance",
    activation_function = "relu",
    signed_inhibitor =  True,
    center = True,
    num_labels = 1,
    )

#student_model = iDistilBertForSequenceClassification.from_pretrained('/shared/Tony/MSc2024/distilbert_init/results/checkpoint-18495', config = student_config)
student_model = iDistilBertForSequenceClassification(
        config=student_config,
    )
initialized_weights = torch.load('/shared/Tony/MSc2024/distilbert_init/models/hiddenstates4_center_inhibitor_init.pth')
student_model.load_state_dict(initialized_weights, strict=False)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
student_model.to(device)


/tmp/ipykernel_4054245/1126949511.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  initialized_weights = torch.load('/shared/Tony/MSc2024/distilbert_init/models/hiddenst

iDistilBertForSequenceClassification(
  (distilbert): iDistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): iTransformer(
      (layer): ModuleList(
        (0-5): 6 x iTransformerBlock(
          (attention): iMultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=Fal

In [ ]:
from transformers import DistilBertForSequenceClassification, AutoModelForSequenceClassification, DistilBertConfig, DataCollatorWithPadding
from iDistilbert import iDistilBertForSequenceClassification
student_id = "distilbert/distilbert-base-uncased"
student_config = DistilBertConfig(    
    distance_metric = "cosine_distance",
    activation_function = "softmax",
    signed_inhibitor =  False,
    alpha = 0,
    center = False,
    output_contexts = False,
)

test_config = DistilBertConfig(         
            num_labels = num_labels,
)
    
student_model = DistilBertForSequenceClassification.from_pretrained(
        student_id,
        config = test_config
    )

#initialized_weights = torch.load('/shared/Tony/MSc2024/distilbert_init/models/hiddenstates_inhibitor_init.pth')
#student_model.load_state_dict(initialized_weights, strict=False)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
student_model.to(device)


In [4]:
import evaluate
import numpy as np

# Load the evaluation metric
pearson_metric = evaluate.load("glue", "stsb")

# Define the compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Clip predictions between 0 and 5 as STS-B labels are in this range
    predictions = np.clip(predictions, 0, 5)
    # Compute Pearson correlation
    return pearson_metric.compute(predictions=predictions, references=labels)


In [5]:
EPOCHS = 4
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

            
training_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs = EPOCHS,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    learning_rate = LEARNING_RATE,
    logging_dir = './logs',
    load_best_model_at_end= True,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps = 20,
    report_to=['tensorboard'],
    lr_scheduler_type="cosine",
    #gradient_accumulation_steps=2,
    weight_decay = 0.01,
)

trainer = Trainer(
    model=student_model,                         
    args=training_args,                  
    train_dataset=tokenized_datasets['train'],         
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics = compute_metrics,
    tokenizer = tokenizer,
    data_collator = data_collator,
)


In [6]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Pearson,Spearmanr
1,1.423800,2.744536,0.466484,0.451165
2,0.814600,1.155776,0.789493,0.807562
3,0.627900,0.838729,0.828929,0.831695
4,0.417800,0.747226,0.833771,0.833630


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=1440, training_loss=1.058040302991867, metrics={'train_runtime': 1092.344, 'train_samples_per_second': 21.052, 'train_steps_per_second': 1.318, 'total_flos': 3046167246065664.0, 'train_loss': 1.058040302991867, 'epoch': 4.0})

In [7]:
for name, param in student_model.named_parameters():
    if 'gamma' in name:
        print(f"{name} (gamma): {param}")
    if 'alpha' in name:
        print(f"{name} (alpha): {param}")
    if 'beta' in name:
        print(f"{name} (beta): {param}")

distilbert.transformer.layer.0.attention.alpha (alpha): Parameter containing:
tensor(0.6472, device='cuda:0', requires_grad=True)
distilbert.transformer.layer.0.attention.beta (beta): Parameter containing:
tensor(0.8316, device='cuda:0', requires_grad=True)
distilbert.transformer.layer.0.attention.gamma (gamma): Parameter containing:
tensor(0.6950, device='cuda:0', requires_grad=True)
distilbert.transformer.layer.1.attention.alpha (alpha): Parameter containing:
tensor(0.6092, device='cuda:0', requires_grad=True)
distilbert.transformer.layer.1.attention.beta (beta): Parameter containing:
tensor(0.9237, device='cuda:0', requires_grad=True)
distilbert.transformer.layer.1.attention.gamma (gamma): Parameter containing:
tensor(0.8291, device='cuda:0', requires_grad=True)
distilbert.transformer.layer.2.attention.alpha (alpha): Parameter containing:
tensor(0.6021, device='cuda:0', requires_grad=True)
distilbert.transformer.layer.2.attention.beta (beta): Parameter containing:
tensor(0.8741, dev